In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d jangedoo/utkface-new

 94% 312M/331M [00:02<00:00, 110MB/s]
100% 331M/331M [00:02<00:00, 155MB/s]


In [4]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [5]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [7]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
len(age)

23708

In [9]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [10]:
df.shape

(23708, 3)

In [11]:
df.head()

,age,gender,img
0,1,1,1_1_2_20161219154612988.jpg.chip.jpg
1,18,1,18_1_3_20170104221856991.jpg.chip.jpg
2,26,0,26_0_1_20170113175136531.jpg.chip.jpg
3,27,1,27_1_2_20170116165628229.jpg.chip.jpg
4,26,1,26_1_0_20170116153823864.jpg.chip.jpg


In [12]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [13]:
train_df.shape

(20000, 3)

In [14]:
test_df.shape

(3708, 3)

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [20]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [21]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 1s 0us/step


In [22]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [23]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [24]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [25]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
625/625 [==============================] - 251s 379ms/step - loss: 100.8367 - age_loss: 15.5545 - gender_loss: 0.8614 - age_mae: 15.5545 - gender_accuracy: 0.5109 - val_loss: 83.1286 - val_age_loss: 14.5501 - val_gender_loss: 0.6927 - val_age_mae: 14.5501 - val_gender_accuracy: 0.5189
Epoch 2/10
625/625 [==============================] - 231s 370ms/step - loss: 83.9085 - age_loss: 15.0688 - gender_loss: 0.6954 - age_mae: 15.0688 - gender_accuracy: 0.5221 - val_loss: 83.3106 - val_age_loss: 14.7470 - val_gender_loss: 0.6926 - val_age_mae: 14.7470 - val_gender_accuracy: 0.5189
Epoch 3/10
625/625 [==============================] - 233s 372ms/step - loss: 83.8906 - age_loss: 14.9158 - gender_loss: 0.6967 - age_mae: 14.9158 - gender_accuracy: 0.5222 - val_loss: 83.2371 - val_age_loss: 14.6779 - val_gender_loss: 0.6925 - val_age_mae: 14.6779 - val_gender_accuracy: 0.5189
Epoch 4/10
625/625 [==============================] - 248s 396ms/step - loss: 83.4565 - age_loss: 14.8591 - gen